# CS394 Module 2

## Preset

### Load required packages

In [ ]:
!uv pip install gradio==5.49.1 openai python-dotenv pydantic

Using Python 3.12.11 environment at: D:\Projects\Digipen\2026Spring\CS394\GenAI-Learning\.venv
Audited 3 packages in 23ms


### Load Openrouter API Key

In [2]:
from google.colab import userdata
OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')

ModuleNotFoundError: No module named 'google'

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")

### Initialize OpenAI

In [5]:
import openai
# Initialize OpenAI client
client = openai.OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=OPENROUTER_API_KEY,
)

## Implement Gradio UI

In [45]:
system_prompt = (
"""# System

You are a professional travel consultant and local expert with 20 years of experience. Your goal is to plan a 'feasible and efficient' travel itinerary that perfectly fits the user's preferences, budget, and schedule.

## Guidelines

This is a single-turn session concluded within a single conversation. Even if the user's query is unclear, you must find the optimal pattern and propose a complete plan to the user.
""")

structure_prompt = (
"""<system>Now, create a detailed travel itinerary based on the user's preferences.</system>"""
)

In [46]:
from pydantic import BaseModel, Field
from datetime import datetime
from enum import Enum

class WaypointType(str, Enum):
    RESTAURANT = "restaurant"
    HOTEL = "hotel"
    TOURIST_SPOT = "tourist_spot"

class Waypoint(BaseModel):
    type: WaypointType
    time_from: datetime = Field(description="Estimated starting time of the activity")
    time_to: datetime = Field(description="Estimated ending time of the activity")
    location: str
    activity: str

class TravelItinerary(BaseModel):
    waypoints: list[Waypoint]


In [48]:
import gradio as gr

def chat_with_streaming(message, history, provider):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    
    # Stream the response
    stream = client.chat.completions.create(
        model=provider,
        messages=messages,
        stream=True,
    )
    
    response_text = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            token = chunk.choices[0].delta.content
            response_text += token
            yield response_text, {}

    # After streaming is done, request structured response (I know it's better to use tool_calls, but this is just for demo...)
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}] + [{"role": "assistant", "content": response_text}] + [{"role": "user", "content": structure_prompt}]

    completion  = client.chat.completions.parse(
        model=provider,
        messages=messages,
        response_format=TravelItinerary
    )
    
    yield response_text, completion.choices[0].message.parsed

# Create streaming chat interface

provider_list = ["openai/gpt-5.2-chat", "anthropic/claude-sonnet-4.5", "anthropic/claude-opus-4.5", "google/gemini-3-flash-preview", "google/gemini-3-pro-preview", "deepseek/deepseek-v3.2-speciale", "xiaomi/mimo-v2-flash:free"]
# Be careful to use Claude "The Money Eater Has Invisible Something" Opus 4.5


with gr.Blocks() as demo:
    with gr.Row():
        # ===== Left: A =====
        with gr.Column(scale=1):
            providerA = gr.Dropdown(
                choices=provider_list,
                label="Provider A",
                value=provider_list[0],
            )
            waypointsA = gr.JSON(label="Waypoints A")

            gr.ChatInterface(
                fn=chat_with_streaming,
                type="messages",
                additional_outputs=[waypointsA],
                additional_inputs=[providerA],
            )

        # ===== Right: B =====
        with gr.Column(scale=1):
            providerB = gr.Dropdown(
                choices=provider_list,
                label="Provider B",
                value=provider_list[0],
            )
            waypointsB = gr.JSON(label="Waypoints B")
            gr.ChatInterface(
                fn=chat_with_streaming,
                type="messages",
                additional_outputs=[waypointsB],
                additional_inputs=[providerB],
            )


demo.launch()

* Running on local URL:  http://127.0.0.1:7894
* To create a public link, set `share=True` in `launch()`.
